# Importation des Package

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import bs4
import re
import copy
import collections as co
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np


# Webscrapping du top 10 netflix journalier

#### Fonction qui récupère le dataframe quand on lui donne la date sous forme string "aaaa/mm/jj", url est un argument conditionnel.

In [66]:
def recup_top_netflix_gen(date,url="https://www.the-numbers.com/home-market/netflix-daily-chart/"):
    
    url_enrichi = url+date
    from urllib.request import Request, urlopen
    req_text = urlopen(Request(url_enrichi, headers={'User-Agent': 'Mozilla/5.0'})).read()
    page = bs4.BeautifulSoup(req_text, "lxml")
    #print (page)

    """On récupère les données------------------"""
    #tableau=page.find("table",{'class' : 'chart full-width'})
    tableau=page.find("div",{'id':'page_filling_chart'})
    #print(tableau)
    #table_body=tableau.find('tbody')
    #print(table_body)
    rows = tableau.find_all('tr')
    #print (rows)

  
    """On récupére tous les élements correspondant au titre et à la note et On crée un dictionnaire dans lequel on met le contenu des lignes nettoyées
    L'index est le titre, la valeur la note.
    --------------"""
    new_rows=[]#on crée une liste vide dans laquelle on va mettre le contenu des lignes
    new_cols = 10*[[]]
    dico_top_netflix = dict()
    i = 0
    for row in rows:
        cols = row.find_all('td') # c'est une liste
        cols = [ele.text.strip() for ele in cols]
        #print (cols)
        if len(cols)> 0:
            dico_top_netflix[cols[0]] = cols[1:]
    #print(dico_top_netflix)    

    """"""

    """On crée ENFIN le dataframe"""
    df_top_netflix_gen = pd.DataFrame.from_dict(dico_top_netflix,orient='index')

    """On rajoute une index différente du titre"""
    for row in rows:
        cols = row.find_all('th')
        #print(cols)
        if len(cols) > 0 : 
            cols = [ele.get_text(separator=' ').strip().title() for ele in cols]
            columns_title = cols
    #print (columns_title)
    

    """On change le nom des colonnes"""
    df_top_netflix_gen.columns=columns_title[1:]
    
    """On vire les colonnes qui ne nous intéresse pas"""
    del(df_top_netflix_gen['Yd'])
    del(df_top_netflix_gen['Lw'])
    del(df_top_netflix_gen['Watch Now'])
    
    
    """On rajoute une colonne 'Measure Date' """
    df_top_netflix_gen['Measure Date']= [date]*df_top_netflix_gen.shape[0]
    #print ("Graph au "+ date)
    return (df_top_netflix_gen)

## Concaténer les tableaux

Liste de toutes les dates valides

In [67]:
liste_dates_valides=[]
for m in range (3,13):
    m=str(m)
    if len(m)==1:
        m = "0"+m
    for d in range (1,32):
        d=str(d)
        if len(d)==1:
            d="0"+d
        date = "2020/"+m+"/"+d
        liste_dates_valides.append(date)

#On supprime les dates sans valeurs
for i in ['2020/03/01', '2020/03/02', '2020/03/03', '2020/03/04', '2020/03/05', '2020/03/06', '2020/03/07', '2020/03/08', 
          '2020/03/09', '2020/03/10', '2020/03/11', '2020/03/12', '2020/03/13', '2020/03/14', '2020/03/15', '2020/03/16', 
          '2020/03/17', '2020/03/18', '2020/03/19', '2020/03/20', '2020/03/21', '2020/03/22', '2020/03/23', '2020/03/28', 
          '2020/03/29', '2020/04/04', '2020/04/05', '2020/04/11', '2020/04/12', '2020/04/18', '2020/04/19', '2020/04/25', 
          '2020/04/26', '2020/08/09', '2020/08/30', '2020/12/10', '2020/12/11', '2020/12/12', '2020/12/13', '2020/12/14',
          '2020/12/15', '2020/12/16', '2020/12/17', '2020/12/18', '2020/12/19', '2020/12/20', '2020/12/21', '2020/12/22', 
          '2020/12/23', '2020/12/24', '2020/12/25', '2020/12/26', '2020/12/27', '2020/12/28', '2020/12/29', '2020/12/30', 
          '2020/12/31', '2020/04/31', '2020/06/31', '2020/09/31', '2020/11/31', '2020/08/06']:
    liste_dates_valides.remove(i)


#### essai

In [68]:
df_06_05=recup_top_netflix_gen("2020/06/05")
df_06_06=recup_top_netflix_gen("2020/06/06")
DF=pd.concat([df_06_05, df_06_06])
DF.sort_values('Measure Date',ascending=False)


,Title,Type,Netflix Excl.,Netflix Release Date,Days In Top 10,Viewer- Ship Score,Measure Date
8,Clueless,Movie,,,4,11,2020/06/06
7,The Help,Movie,,,4,17,2020/06/06
6,Sweet Magnolias,TV Show,Yes,"May 19, 2020",18,142,2020/06/06
5,Avatar the Last Airbender,TV Show,,,19,145,2020/06/06
4,Cloudy with a Chance of Mea…,Movie,,,2,7,2020/06/06
3,The Last Days of American C…,Movie,Yes,"Jun 5, 2020",1,6,2020/06/06
2,Fuller House,TV Show,Yes,,4,31,2020/06/06
1,Jeffrey Epstein: Filthy Rich,TV Show,Yes,"May 27, 2020",10,91,2020/06/06
2,Jeffrey Epstein: Filthy Rich,TV Show,Yes,"May 27, 2020",9,83,2020/06/05
1,Space Force,TV Show,Yes,"May 29, 2020",7,70,2020/06/05


In [69]:
df_mars=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_mars')
df_avril=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_avril')
df_mai=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_mai')
df_juin=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_juin')
df_juillet=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_juillet')
df_aout=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_aout')
df_septembre=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_septembre')
df_octobre=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_octobre')
df_novembre=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_novembre')
df_decembre=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_decembre')


In [71]:
def sortby(df):
    return df.sort_values(['Title','Measure Date'],ascending=(True,False))
#sortby(df_mars)

#### Création de dico_score

In [72]:
liste_df = [df_decembre,df_novembre,df_octobre,df_septembre,df_aout,df_juillet,df_juin,df_mai,df_avril,df_mars]
dico_score=dict()

for df_moi in liste_df:
    for i in range(sortby(df_moi).shape[0]):
        if sortby(df_moi)['Title'][i] not in dico_score:
            dico_score[sortby(df_moi)['Title'][i]]=sortby(df_moi)['Viewer- Ship Score'][i]

dico_score


{'Holidate': 40,
 "The Queen's Gambit": 77,
 'Rogue City': 11,
 'His House': 8,
 'Hubie Halloween': 140,
 'Secrets of the Saqqara Tomb': 15,
 'Over the Moon': 64,
 'The Haunting of Bly Manor': 177,
 'Blood of Zeus': 24,
 'Barbarians': 56,
 'Virgin River': 49,
 'The Christmas Chronicles 2': 66,
 'The 2nd': 16,
 'The Crown': 144,
 'Dr. Seussâ\x80\x99 The Grinch': 89,
 'Hillbilly Elegy': 56,
 'La belva': 17,
 'How the Grinch Stole Christmas': 8,
 'Cocomelon': 160,
 'Peppermint': 9,
 'Rust Creek': 7,
 'Alien Worlds': 3,
 'Marauders': 8,
 'Selena: The Series': 10,
 'Big Mouth': 9,
 'Ava': 4,
 'Triple 9': 6,
 'Knock Knock': 8,
 'Grand Army': 36,
 'Mile 22': 4,
 'Snowden': 3,
 'Easy A': 6,
 "Chappelle's Show": 3,
 'Operation Christmas Drop': 9,
 'The Impossible': 1,
 'The Great British Baking Show': 33,
 'Midnight at the Magnolia': 1,
 'Dash & Lily': 6,
 '60 Days In': 1,
 'Trial 4': 5,
 'The Liberator': 5,
 'Jingle Jangle: A Christmas …': 9,
 'American Horror Story': 6,
 'La vita davanti a sÃ

Convertit le score en intégral

In [73]:
for cle in dico_score.keys():
    u=dico_score[cle]
    dico_score[cle] = int(u)
dico_score

{'Holidate': 40,
 "The Queen's Gambit": 77,
 'Rogue City': 11,
 'His House': 8,
 'Hubie Halloween': 140,
 'Secrets of the Saqqara Tomb': 15,
 'Over the Moon': 64,
 'The Haunting of Bly Manor': 177,
 'Blood of Zeus': 24,
 'Barbarians': 56,
 'Virgin River': 49,
 'The Christmas Chronicles 2': 66,
 'The 2nd': 16,
 'The Crown': 144,
 'Dr. Seussâ\x80\x99 The Grinch': 89,
 'Hillbilly Elegy': 56,
 'La belva': 17,
 'How the Grinch Stole Christmas': 8,
 'Cocomelon': 160,
 'Peppermint': 9,
 'Rust Creek': 7,
 'Alien Worlds': 3,
 'Marauders': 8,
 'Selena: The Series': 10,
 'Big Mouth': 9,
 'Ava': 4,
 'Triple 9': 6,
 'Knock Knock': 8,
 'Grand Army': 36,
 'Mile 22': 4,
 'Snowden': 3,
 'Easy A': 6,
 "Chappelle's Show": 3,
 'Operation Christmas Drop': 9,
 'The Impossible': 1,
 'The Great British Baking Show': 33,
 'Midnight at the Magnolia': 1,
 'Dash & Lily': 6,
 '60 Days In': 1,
 'Trial 4': 5,
 'The Liberator': 5,
 'Jingle Jangle: A Christmas …': 9,
 'American Horror Story': 6,
 'La vita davanti a sÃ

### Fonction qui convertit un dico en df

In [74]:
def dict_to_df(dico,columns_titles,sortby='Score'):
    df = pd.DataFrame.from_dict(dico,orient='index')
    df.columns=columns_titles
    return df.sort_values(sortby,ascending=False)

In [75]:
df_score = dict_to_df(dico_score,['Score'])

#### Le dico pour aussi avoir le type

In [76]:
liste_df = [df_decembre,df_novembre,df_octobre,df_septembre,df_aout,df_juillet,df_juin,df_mai,df_avril,df_mars]
dico_type_score=dict()

for df_moi in liste_df:
    for i in range(sortby(df_moi).shape[0]):
        if sortby(df_moi)['Title'][i] not in dico_type_score:
            dico_type_score[sortby(df_moi)['Title'][i]]=[sortby(df_moi)['Type'][i],sortby(df_moi)['Viewer- Ship Score'][i]]

#dico_type_score

In [77]:
for cle in dico_type_score.keys():
    u=dico_type_score[cle][1]
    dico_type_score[cle][1] = int(u)
#dico_type_score


In [78]:
df_type_score=dict_to_df(dico_type_score,['Type','Score'])
df_type_score[15:50]

,Type,Score
Sweet Magnolias,TV Show,110
Riverdale,TV Show,103
The Order,TV Show,101
The Last Dance,TV Show,98
Lucifer,TV Show,95
Jurassic World: Camp Cretac…,TV Show,91
Dr. Seussâ The Grinch,Movie,89
The Queen's Gambit,TV Show,77
Enola Holmes,Movie,71
"Tiger King: Murder, Mayhem …",TV Show,70


Combien on a de séries ?

In [79]:
df_type_score[['Type']].isin(['Movie']).sum() #145
df_type_score[['Type']].isin(['TV Show']).sum() #141
df_type_score[['Type']].isna().sum() #0 -> manque 3 lignes ?

Type    0
dtype: int64

On va récup les CSV correspondant

In [80]:
gfg_csv_data = df_type_score.to_csv('type_score.csv', index = True) 
#print('\nCSV String:\n', gfg_csv_data) 

Pour avoir NEtflix Exclusive

In [81]:
liste_df = [df_decembre,df_novembre,df_octobre,df_septembre,df_aout,df_juillet,df_juin,df_mai,df_avril,df_mars]
dico_type_NE_score=dict()

for df_moi in liste_df:
    for i in range(sortby(df_moi).shape[0]):
        if sortby(df_moi)['Title'][i] not in dico_type_NE_score:
            dico_type_NE_score[sortby(df_moi)['Title'][i]]=[sortby(df_moi)['Type'][i],sortby(df_moi)['Netflix Excl.'][i],sortby(df_moi)['Viewer- Ship Score'][i]]

dico_type_NE_score

{'Holidate': ['Movie', 'Yes', 40],
 "The Queen's Gambit": ['TV Show', 'Yes', 77],
 'Rogue City': ['Movie', 'Yes', 11],
 'His House': ['Movie', 'Yes', 8],
 'Hubie Halloween': ['Movie', 'Yes', 140],
 'Secrets of the Saqqara Tomb': ['Movie', 'Yes', 15],
 'Over the Moon': ['Movie', 'Yes', 64],
 'The Haunting of Bly Manor': ['TV Show', 'Yes', 177],
 'Blood of Zeus': ['TV Show', 'Yes', 24],
 'Barbarians': ['TV Show', 'Yes', 56],
 'Virgin River': ['TV Show', 'Yes', 49],
 'The Christmas Chronicles 2': ['Movie', 'Yes', 66],
 'The 2nd': ['Movie', nan, 16],
 'The Crown': ['TV Show', 'Yes', 144],
 'Dr. Seussâ\x80\x99 The Grinch': ['Movie', nan, 89],
 'Hillbilly Elegy': ['Movie', 'Yes', 56],
 'La belva': ['Movie', 'Yes', 17],
 'How the Grinch Stole Christmas': ['Movie', nan, 8],
 'Cocomelon': ['TV Show', nan, 160],
 'Peppermint': ['Movie', nan, 9],
 'Rust Creek': ['Movie', nan, 7],
 'Alien Worlds': ['TV Show', 'Yes', 3],
 'Marauders': ['Movie', nan, 8],
 'Selena: The Series': ['TV Show', 'Yes', 10]

In [82]:
for cle in dico_type_NE_score.keys():
    u=dico_type_NE_score[cle][2]
    dico_type_NE_score[cle][2] = int(u)
dico_type_NE_score

{'Holidate': ['Movie', 'Yes', 40],
 "The Queen's Gambit": ['TV Show', 'Yes', 77],
 'Rogue City': ['Movie', 'Yes', 11],
 'His House': ['Movie', 'Yes', 8],
 'Hubie Halloween': ['Movie', 'Yes', 140],
 'Secrets of the Saqqara Tomb': ['Movie', 'Yes', 15],
 'Over the Moon': ['Movie', 'Yes', 64],
 'The Haunting of Bly Manor': ['TV Show', 'Yes', 177],
 'Blood of Zeus': ['TV Show', 'Yes', 24],
 'Barbarians': ['TV Show', 'Yes', 56],
 'Virgin River': ['TV Show', 'Yes', 49],
 'The Christmas Chronicles 2': ['Movie', 'Yes', 66],
 'The 2nd': ['Movie', nan, 16],
 'The Crown': ['TV Show', 'Yes', 144],
 'Dr. Seussâ\x80\x99 The Grinch': ['Movie', nan, 89],
 'Hillbilly Elegy': ['Movie', 'Yes', 56],
 'La belva': ['Movie', 'Yes', 17],
 'How the Grinch Stole Christmas': ['Movie', nan, 8],
 'Cocomelon': ['TV Show', nan, 160],
 'Peppermint': ['Movie', nan, 9],
 'Rust Creek': ['Movie', nan, 7],
 'Alien Worlds': ['TV Show', 'Yes', 3],
 'Marauders': ['Movie', nan, 8],
 'Selena: The Series': ['TV Show', 'Yes', 10]

In [84]:
df_type_NE_score=dict_to_df(dico_type_NE_score,['Type','Netflix Exclusive','Score'])
df_type_NE_score['Netflix Exclusive']=df_type_NE_score['Netflix Exclusive'].fillna('No')
gfg_csv_data = df_type_NE_score.to_csv('type_NE_score.csv', index = True) 